In [7]:
%reload_ext autoreload
%autoreload 2

import os
from enum import Enum
from typing import Any, Dict, Optional
from typing_extensions import Literal
from pathlib import Path
from tqdm.notebook import tqdm 
import cv2
import numpy as np
import shutil
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

import rawpy
import lensfunpy

from funcs import read_process_img, get_files

# Conference Room
root_dir = Path("../../data/real/ConferenceRoom")
times = np.array([1/15, 1/30, 1/8, 1/60, 1/4, 1/125, 1/2, 1/250, 1], dtype=np.float32)
# METHOD = "superglue"
METHOD = "colmap"
calib_dir = root_dir.parent / "Sony/calib_raw"

image_dir = root_dir / "raw_images"
calib_paths = get_files(calib_dir)
image_paths = get_files(image_dir)

# Get calibration files

In [10]:
index = 4
p = image_paths[index]
img_max = 2**14 - 1

with rawpy.imread(str(p)) as raw:
    raw_processed = cv2.demosaicing(raw.raw_image, code=cv2.COLOR_BAYER_BG2BGR)
    
height, width, _ = raw_processed.shape

db = lensfunpy.Database()
# Sony Setting
cam = db.find_cameras("Sony", "ILCE-7M3")[0]
lens = db.find_lenses(cam, "Sony", "FE 24-105mm")[0]
focal_length, aperture, distance = 24, 20, 0.5
# Canon Setting
# cam = db.find_cameras("Canon", "Canon EOS 5D Mark III")[0]
# lens = db.find_lenses(cam, "Canon", "Canon EF 24-70mm f/2.8L II USM")[0]
# focal_length, aperture, distance = 24, 19, 2.2

mod = lensfunpy.Modifier(lens, cam.crop_factor, width, height)
mod.initialize(focal_length, aperture, distance, scale=1.0, pixel_format=np.float32)

# Sony
white_balance = np.array([2.6657157, 1.0 ,1.9722375], dtype=np.float32)
# # Canon
# white_balance = np.array([2.5430965, 1., 1.5503172], dtype=np.float32)

In [11]:
# !jupyter nbextension enable --py widgetsnbextension

# downsample images, resolution is too high
downsample_scale = 8
checkerboard_size = (16, 13)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 500, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((np.prod(checkerboard_size),3), np.float32)
objp[:,:2] = np.mgrid[0:checkerboard_size[0],0:checkerboard_size[1]].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
# Main loop for extracting corners
for calib_img_path in tqdm(calib_paths):
    assert Path(calib_img_path).exists()
    img = read_process_img(calib_img_path, mod=mod) * white_balance
    # Linear to grayscale conversion with scaling
    img_gray = img[...,2] * 0.2126 + img[...,1] * 0.7152 + img[...,0] * 0.0722
#     img_gray = np.average(img, axis=-1)
    img_gray /= np.max(img_gray)
    img_gray_resized = cv2.resize(
        img_gray, 
        dsize=(img.shape[1] // downsample_scale, img.shape[0] // downsample_scale), 
        interpolation=cv2.INTER_AREA)
    ret, corners_resized = cv2.findChessboardCorners(
        (img_gray_resized * 255).astype(np.uint8), checkerboard_size, None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        corners = corners_resized * downsample_scale
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(img_gray, np.floor(corners), (40,40), (-1,-1), criteria)
        imgpoints.append(corners2)
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_gray.shape[::-1], None, None)
print("Reprojection error:", ret)

# fig = plt.figure(figsize=(8,6))
# plt.imshow(img_gray)
# plt.plot(corners2[:,0,0], corners2[:,0,1], "ro")

  0%|          | 0/59 [00:00<?, ?it/s]

Reprojection error: 0.725095331733294


# Prepare folders

In [13]:
h, w = img_gray.shape[:2]
exr_resize_shape = (1152, 768)
upsample_factor = int(np.max(np.ceil(img_gray.shape[-1::-1] / np.array(exr_resize_shape))))

w_new, h_new = np.array(exr_resize_shape) * upsample_factor
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(
    mtx, dist, (w,h), 
    0, # Contain only the valid pixels
    (w_new,h_new)
)
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w_new,h_new), cv2.CV_32F)

# Create directories
exr_output_dir = root_dir / "merged_images"
exr_output_dir.mkdir(exist_ok=True)
png_output_dir = root_dir / "png_images"
png_output_dir.mkdir(exist_ok=True)
pose_output_dir = root_dir / "pose_images"
pose_output_dir.mkdir(exist_ok=True)
# Setup the image scales and output brightness
pose_resize_scale = 4
png_exposure_scale = 1.5
depth_normal_exposure_scale = 2.0
# Setup the params for combining images
apply_median_blur = False
times_min_index = np.argmin(times)
times_max_index = np.argmax(times)
exposure_target = times[-1]
# Depth Normal estimation
estimate_depth = True
estimate_normals = True

# Process RAW images

In [17]:
from multiprocessing import Pool
import time
import copy

def process_bracket(_):
# for index in tqdm(range(len(image_paths)//len(times))):
#     index = 83
    index = _
    
    exr_path = exr_output_dir / f"img_{str(index).zfill(4)}.exr"
    pose_path = pose_output_dir / f"img_{str(index).zfill(4)}.png"
    png_path = png_output_dir / f"img_{str(index).zfill(4)}.png"
    # Get the correct image index
    index *= len(times)
    
    # Read and correct images
    images = []
    for i, time in enumerate(times):
        images.append(read_process_img(str(image_paths[index]), mod=mod))
        index += 1
    images = np.array(images)
    
    # Calculate hat function for averaging the values
    weights = np.minimum(images, 1.0 - images)
    weights[times_max_index][images[times_max_index]<0.5] = 1.0
    weights[times_min_index][images[times_min_index]>0.5] = 1.0
    weights /= np.sum(weights, axis=0)
    
    # Scale the images to same exposure
    images_scaled = images * (exposure_target / times)[:, np.newaxis, np.newaxis, np.newaxis]
    # Combine the images
    imageHDR = np.sum(images_scaled * weights, axis=0) * white_balance
    if apply_median_blur:
        imageHDR = cv2.medianBlur(imageHDR, ksize=3)
        
    # Undistort hdr image
    imageHDR = cv2.remap(imageHDR, mapx, mapy, cv2.INTER_LINEAR)
    
    # Write exr image for reconstruction
    output_img_resized = cv2.resize(
        imageHDR, 
        dsize=exr_resize_shape, 
        interpolation=cv2.INTER_AREA)
    cv2.imwrite(str(exr_path), output_img_resized)
    png_image = np.clip(output_img_resized * png_exposure_scale, a_min=0, a_max=1) **(1/2.2)
    png_image = (png_image * 255).astype(np.uint8)
    cv2.imwrite(str(png_path), png_image)
    
    # Write png pose image for colmap poses
    pose_image = cv2.resize(
        imageHDR, 
        dsize=(exr_resize_shape[0]*pose_resize_scale, exr_resize_shape[1]*pose_resize_scale), 
        interpolation=cv2.INTER_AREA)
    pose_image = np.clip(pose_image * png_exposure_scale, a_min=0, a_max=1) **(1/2.2)
    pose_image = (pose_image * 255).astype(np.uint8)
    cv2.imwrite(str(pose_path), pose_image)
    
tic = time.time()
p = Pool(processes=16)
index_list = range(len(image_paths)//len(times))
_list = [index for index in index_list]
print(len(_list), len(index_list))
list(tqdm(p.imap_unordered(process_bracket, _list), total=len(index_list)))
p.close()
p.join()
print('==== ...DONE. Took %.2f seconds'%(time.time() - tic))

# [!!!!] Tried this on a Mac due to some errors with multiprocessing, but never sorted it out....
# ...as a result, the notebook is only tested on Ubuntu so far

# params = {
#     'exr_output_dir': exr_output_dir, 
#     'pose_output_dir': pose_output_dir, 
#     'png_output_dir': png_output_dir, 
#     'times': times, 
#     'image_paths': image_paths, 
#     'times_max_index': times_max_index, 
#     'times_min_index': times_min_index, 
#     'exposure_target': exposure_target, 
#     'white_balance': white_balance, 
#     'apply_median_blur': apply_median_blur, 
#     'mapx': mapx, 
#     'mapy': mapy, 
#     'exr_resize_shape': exr_resize_shape, 
#     'png_exposure_scale': png_exposure_scale, 
#     'pose_resize_scale': pose_resize_scale, 
#     'mod': mod,
#     # 'height': height, 
#     # 'width': width,
# }

# tic = time.time()
# p = Pool(processes=1)
# index_list = range(len(image_paths)//len(times))
# _list = [(index, params) for index in index_list]
# print(len(_list), len(index_list))
# list(tqdm(p.imap_unordered(process_bracket, _list), total=len(index_list)))
# p.close()
# p.join()
# print('==== ...DONE. Took %.2f seconds'%(time.time() - tic))

284 284


  0%|          | 0/284 [00:00<?, ?it/s]

Process ForkPoolWorker-13:
Traceback (most recent call last):
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-17:

KeyboardInterrupt

Traceback (most recent call last):
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/tmp/ipykernel_30666/3339153429.py", line 24, in process_bracket
    weights = np.minimum(images, 1.0 - images)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Process ForkPoolWorker-11:
Traceback (most recent call last):
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ruizhu/anaconda3/

Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/tmp/ipykernel_30666/3339153429.py", line 32, in process_bracket
    imageHDR = np.sum(images_scaled * weights, axis=0) * white_balance
  File "<__array_function__ internals>", line 177, in sum
KeyboardInterrupt
Process ForkPoolWorker-20:
Traceback (most recent call last):
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ruizhu/anaconda3/envs/or-py310/lib/python3.10/multiprocessing/process.py", line 108, in

## Colmap/Superglue reconstruction

In [14]:
from nerfstudio.process_data.colmap_utils import get_colmap_version, get_vocab_tree
from nerfstudio.utils.rich_utils import status
from nerfstudio.utils.scripts import run_command
from rich.console import Console
CONSOLE = Console(width=120)

In [15]:
class CameraModel(Enum):
    """Enum for camera types."""

    OPENCV = "OPENCV"
    OPENCV_FISHEYE = "OPENCV_FISHEYE"
    PINHOLE = "PINHOLE"
    SIMPLE_PINHOLE = "SIMPLE_PINHOLE"


def run_colmap(
    image_dir: Path,
    colmap_dir: Path,
    camera_model: CameraModel,
    camera_mask_path: Optional[Path] = None,
    camera_params = None,
    gpu: bool = True,
    verbose: bool = False,
    matching_method: Literal["vocab_tree", "exhaustive", "sequential"] = "vocab_tree",
    colmap_cmd: str = "colmap",
):
    camera_params = '"' + ",".join([i.astype(str) for i in camera_params]) + '"'
    colmap_version = get_colmap_version(colmap_cmd)

    colmap_database_path = colmap_dir / "database.db"
    if colmap_database_path.exists():
        # Can't use missing_ok argument because of Python 3.7 compatibility.
        colmap_database_path.unlink()

    # Feature extraction
    feature_extractor_cmd = [
        f"{colmap_cmd} feature_extractor",
        f"--database_path {colmap_dir / 'database.db'}",
        f"--image_path {image_dir}",
        "--ImageReader.single_camera 1",
        f"--ImageReader.camera_model {camera_model.value}",
        f"--ImageReader.camera_params {camera_params}",
        f"--SiftExtraction.use_gpu {int(gpu)}",
    ]
    if camera_mask_path is not None:
        feature_extractor_cmd.append(f"--ImageReader.camera_mask_path {camera_mask_path}")
    feature_extractor_cmd = " ".join(feature_extractor_cmd)
    with status(msg="[bold yellow]Running COLMAP feature extractor...", spinner="moon", verbose=verbose):
        run_command(feature_extractor_cmd, verbose=verbose)

    CONSOLE.log("[bold green]:tada: Done extracting COLMAP features.")

    # Feature matching
    feature_matcher_cmd = [
        f"{colmap_cmd} {matching_method}_matcher",
        f"--database_path {colmap_dir / 'database.db'}",
        f"--SiftMatching.use_gpu {int(gpu)}",
    ]
    if matching_method == "vocab_tree":
        vocab_tree_filename = get_vocab_tree()
        feature_matcher_cmd.append(f"--VocabTreeMatching.vocab_tree_path {vocab_tree_filename}")
    feature_matcher_cmd = " ".join(feature_matcher_cmd)
    with status(msg="[bold yellow]Running COLMAP feature matcher...", spinner="runner", verbose=verbose):
        run_command(feature_matcher_cmd, verbose=verbose)
    CONSOLE.log("[bold green]:tada: Done matching COLMAP features.")

    # Bundle adjustment
    sparse_dir = colmap_dir / "sparse"
    sparse_dir.mkdir(parents=True, exist_ok=True)
    mapper_cmd = [
        f"{colmap_cmd} mapper",
        f"--database_path {colmap_dir / 'database.db'}",
        f"--image_path {image_dir}",
        f"--output_path {sparse_dir}",
        "--Mapper.ba_refine_focal_length 0",
        "--Mapper.ba_refine_extra_params 0",
    ]
    if colmap_version >= 3.7:
        mapper_cmd.append("--Mapper.ba_global_function_tolerance 1e-6")

    mapper_cmd = " ".join(mapper_cmd)

    with status(
        msg="[bold yellow]Running COLMAP bundle adjustment... (This may take a while)",
        spinner="circle",
        verbose=verbose,
    ):
        run_command(mapper_cmd, verbose=verbose)
    CONSOLE.log("[bold green]:tada: Done COLMAP bundle adjustment.")
    with status(msg="[bold yellow]Refine intrinsics...", spinner="dqpb", verbose=verbose):
        bundle_adjuster_cmd = [
            f"{colmap_cmd} bundle_adjuster",
            f"--input_path {sparse_dir}/0",
            f"--output_path {sparse_dir}/0",
            "--BundleAdjustment.refine_principal_point 0",
            "--BundleAdjustment.refine_focal_length 0",
            "--BundleAdjustment.refine_extra_params 0",
        ]
        run_command(" ".join(bundle_adjuster_cmd), verbose=verbose)
    CONSOLE.log("[bold green]:tada: Done refining intrinsics.")

In [16]:
from hloc import (
    extract_features,
    match_features,
    pairs_from_exhaustive,
    pairs_from_retrieval,
    reconstruction,
)
import pycolmap

def run_hloc(
    image_dir: Path,
    colmap_dir: Path,
    camera_model: CameraModel,
    camera_params = None,
    verbose: bool = False,
    matching_method: Literal["vocab_tree", "exhaustive", "sequential"] = "vocab_tree",
    feature_type: Literal[
        "sift", "superpoint_aachen", "superpoint_max", "superpoint_inloc", "r2d2", "d2net-ss", "sosnet", "disk"
    ] = "superpoint_aachen",
    matcher_type: Literal[
        "superglue", "superglue-fast", "NN-superpoint", "NN-ratio", "NN-mutual", "adalam"
    ] = "superglue",
    num_matched: int = 50,
) -> None:
    camera_params = ",".join([i.astype(str) for i in camera_params])
    
    outputs = colmap_dir
    sfm_pairs = outputs / "pairs-netvlad.txt"
    sfm_dir = outputs / "sparse" / "0"
    features = outputs / "features.h5"
    matches = outputs / "matches.h5"

    retrieval_conf = extract_features.confs["netvlad"]
    feature_conf = extract_features.confs[feature_type]
    matcher_conf = match_features.confs[matcher_type]

    references = [p.relative_to(image_dir).as_posix() for p in image_dir.iterdir()]
    extract_features.main(feature_conf, image_dir, image_list=references, feature_path=features)
    if matching_method == "exhaustive":
        pairs_from_exhaustive.main(sfm_pairs, image_list=references)
    else:
        retrieval_path = extract_features.main(retrieval_conf, image_dir, outputs)
        if num_matched >= len(references):
            num_matched = len(references)
        pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=num_matched)
    match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

    image_options = pycolmap.ImageReaderOptions(  # pylint: disable=c-extension-no-member
        camera_model=camera_model.value,
        camera_params=camera_params
    )
    min_match_score = None
    skip_geometric_verification = False
    
    sfm_dir.mkdir(parents=True, exist_ok=True)
    database = sfm_dir / 'database.db'
    
    reconstruction.create_empty_db(database)
    reconstruction.import_images(
        image_dir, database, pycolmap.CameraMode.SINGLE, references, image_options)
    image_ids = reconstruction.get_image_ids(database)
    reconstruction.import_features(image_ids, database, features)
    reconstruction.import_matches(image_ids, database, sfm_pairs, matches,
                   min_match_score, skip_geometric_verification)
    
    mapper_options = {
      "ba_refine_focal_length": False,
      "ba_refine_principal_point": False,
      "ba_refine_extra_params": False,
    }
    
    if not skip_geometric_verification:
        reconstruction.estimation_and_geometric_verification(
            database, sfm_pairs, verbose)
    rec_out = reconstruction.run_reconstruction(
        sfm_dir, database, image_dir, verbose, mapper_options)
    if rec_out is not None:
        CONSOLE.log(f'Reconstruction statistics:\n{rec_out.summary()}'
                    + f'\n\tnum_input_images = {len(image_ids)}')

In [17]:
colmap_dir = root_dir / "reconstructions" / "reconstruction_colmap"
# colmap_dir = root_dir / "reconstructions" / "reconstruction_hloc"
recon_dir = colmap_dir / "sparse" / "0"
if colmap_dir.exists():
    shutil.rmtree(colmap_dir)
os.makedirs(colmap_dir)

camera_params = np.array(
    [newcameramtx[0,0], newcameramtx[1,1], newcameramtx[0, 2], newcameramtx[1, 2]])
camera_params *= pose_resize_scale / upsample_factor

camera_model = CameraModel.PINHOLE

if METHOD.lower() == 'colmap':
    # USE COLMAP
    run_colmap(
        image_dir=pose_output_dir,
        colmap_dir=colmap_dir,
        camera_model=camera_model,
        camera_params=camera_params,
        gpu=True,
        verbose=True,
        matching_method="exhaustive"
    ); appendix = '_colmap'

elif METHOD.lower() == 'superglue':
    # USE Superglue
    run_hloc(
        image_dir=pose_output_dir,
        colmap_dir=colmap_dir,
        camera_model=camera_model,
        camera_params=camera_params,
        verbose=True,
        matching_method="exhaustive",
        feature_type="superpoint_inloc",
        matcher_type="superglue",
    ); appendix = '_supergloo'
else:
    raise ValueError(f"Unknown method {METHOD}")


Feature extraction

Processed file [1/275]
  Name:            img_0000.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        2162
Processed file [2/275]
  Name:            img_0001.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        2191
Processed file [3/275]
  Name:            img_0002.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        2370
Processed file [4/275]
  Name:            img_0003.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        1545
Processed file [5/275]
  Name:            img_0004.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        1844
Processed file [6/275]
  Name:            img_0005.png
  Dimensions:      4608 x 307

Processed file [46/275]
  Name:            img_0045.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        684
Processed file [47/275]
  Name:            img_0046.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        934
Processed file [48/275]
  Name:            img_0047.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        437
Processed file [49/275]
  Name:            img_0048.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        295
Processed file [50/275]
  Name:            img_0049.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        261
Processed file [51/275]
  Name:            img_0050.png
  Dimensions:      4608 x 3072
  Camera:         

Processed file [92/275]
  Name:            img_0091.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        836
Processed file [93/275]
  Name:            img_0092.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        638
Processed file [94/275]
  Name:            img_0093.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        856
Processed file [95/275]
  Name:            img_0094.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        1140
Processed file [96/275]
  Name:            img_0095.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        275
Processed file [97/275]
  Name:            img_0096.png
  Dimensions:      4608 x 3072
  Camera:        

Processed file [138/275]
  Name:            img_0137.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        98
Processed file [139/275]
  Name:            img_0138.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        70
Processed file [140/275]
  Name:            img_0139.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        83
Processed file [141/275]
  Name:            img_0140.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        126
Processed file [142/275]
  Name:            img_0141.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        111
Processed file [143/275]
  Name:            img_0142.png
  Dimensions:      4608 x 3072
  Camera:      

Processed file [184/275]
  Name:            img_0183.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        768
Processed file [185/275]
  Name:            img_0184.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        798
Processed file [186/275]
  Name:            img_0185.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        397
Processed file [187/275]
  Name:            img_0186.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        1124
Processed file [188/275]
  Name:            img_0187.png
  Dimensions:      4608 x 3072
  Camera:          #1 - PINHOLE
  Focal Length:    3109.96px (Prior)
  Features:        1329
Processed file [189/275]
  Name:            img_0188.png
  Dimensions:      4608 x 3072
  Camera: 

...


Registering image #173 (252)

  => Image sees 56 / 73 points

Pose refinement report
----------------------
    Residuals : 76
   Parameters : 6
   Iterations : 19
         Time : 0.00479913 [s]
 Initial cost : 0.7018 [px]
   Final cost : 0.646366 [px]
  Termination : Convergence

  => Continued observations: 37
  => Added observations: 35

Bundle adjustment report
------------------------
    Residuals : 1826
   Parameters : 317
   Iterations : 15
         Time : 0.0472181 [s]
 Initial cost : 0.601677 [px]
   Final cost : 0.551083 [px]
  Termination : Convergence

  => Merged observations: 0
  => Completed observations: 1
  => Filtered observations: 6
  => Changed observations: 0.007447

Bundle adjustment report
------------------------
    Residuals : 1816
   Parameters : 314
   Iterations : 3
         Time : 0.0120211 [s]
 Initial cost : 0.684694 [px]
   Final cost : 0.679954 [px]
  Termination : Convergence

  => Merged observations: 0
  => Completed observations: 0
  => Filtered 

   7  5.062743e+04    1.26e-03    5.50e+00   1.88e-02   9.98e-01  2.19e+07        1    2.05e-01    1.83e+00
   8  5.062743e+04    3.29e-05    1.05e+00   3.36e-03   9.99e-01  6.56e+07        1    1.97e-01    2.03e+00
   9  5.062743e+04    1.18e-07    1.10e-02   2.05e-04   1.00e+00  1.97e+08        1    1.87e-01    2.22e+00


Bundle adjustment report
------------------------
    Residuals : 155070
   Parameters : 54245
   Iterations : 10
         Time : 2.22579 [s]
 Initial cost : 0.571898 [px]
   Final cost : 0.571385 [px]
  Termination : Convergence

  => Merged observations: 3
  => Completed observations: 0
  => Filtered observations: 3
  => Changed observations: 0.000077
  => Filtered images: 0

Initializing with image pair #204 and #203


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  8.839205e+01    0.00e+00    1.16e+05   0.00e+00   0.00e+00  1.00e+04        0    9.05e-04    3.73e-03
   1  

[00:35:18] 🎉 Done COLMAP bundle adjustment.                                                            ]8;id=181489;file:///tmp/ipykernel_4495/3793546177.py\3793546177.py]8;;\:]8;id=612365;file:///tmp/ipykernel_4495/3793546177.py#83\83]8;;\


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.061034e+04    0.00e+00    1.34e+04   0.00e+00   0.00e+00  1.00e+04        0    5.60e-02    3.76e-01

CHOLMOD version 3.0.11, May 4, 2016: Symbolic Analysis: status: OK
  Architecture: Linux
    sizeof(int):      4
    sizeof(SuiteSparse_long):  8
    sizeof(void *):   8
    sizeof(double):   8
    sizeof(Int):      4 (CHOLMOD's basic integer)
    sizeof(BLAS_INT): 4 (integer used in the BLAS)
  Results from most recent analysis:
    Cholesky flop count: 2.9486e+08
    Nonzeros in L:       5.9818e+05
  memory blocks in use:          11
  memory in use (MB):           0.1
  peak memory usage (MB):       1.5
  maxrank:    update/downdate rank:   8
  supernodal control: 1 40 (supernodal if flops/lnz >= 40)
  nmethods:   number of ordering methods to try: 1
    method 0: natural
        flop count: 2.9486e+08
        nnz(L):     5.9818e+05
  OK
   1 

[00:35:24] 🎉 Done refining intrinsics.                                                                 ]8;id=888623;file:///tmp/ipykernel_4495/3793546177.py\3793546177.py]8;;\:]8;id=475604;file:///tmp/ipykernel_4495/3793546177.py#94\94]8;;\

In [18]:
from nerfstudio.utils.io import load_from_json, write_to_json
from nerfstudio.data.utils.colmap_parsing_utils import (
    read_cameras_binary, read_images_binary, qvec2rotmat, 
)
    
from nerfstudio.process_data.colmap_utils import parse_colmap_camera_params

recon_dir = colmap_dir / "sparse" / "0"

cam_id_to_camera = read_cameras_binary(recon_dir / "cameras.bin")
im_id_to_image = read_images_binary(recon_dir / "images.bin")

print(f"{len(im_id_to_image)} frames matched..")

frames = []
for im_id, im_data in im_id_to_image.items():
    # NB: COLMAP uses Eigen / scalar-first quaternions
    # * https://colmap.github.io/format.html
    # * https://github.com/colmap/colmap/blob/bf3e19140f491c3042bfd85b7192ef7d249808ec/src/base/pose.cc#L75
    # the `rotation_matrix()` handles that format for us.

    # TODO(1480) BEGIN use pycolmap API
    # rotation = im_data.rotation_matrix()
    rotation = qvec2rotmat(im_data.qvec)

    translation = im_data.tvec.reshape(3, 1)
    w2c = np.concatenate([rotation, translation], 1)
    w2c = np.concatenate([w2c, np.array([[0, 0, 0, 1]])], 0)
    c2w = np.linalg.inv(w2c)
    # Convert from COLMAP's camera coordinate system (OpenCV) to ours (OpenGL)
    c2w[0:3, 1:3] *= -1
    c2w = c2w[np.array([1, 0, 2, 3]), :]
    c2w[2, :] *= -1

    name = im_data.name.split(".")[0]+".exr"
    name = Path(f"./{exr_output_dir.name}/{name}")
    
#     name = im_data.name.split(".")[0]+".png"
#     name = Path(f"./{png_output_dir.name}/{name}")

    frame = {
        "file_path": name.as_posix(),
        "transform_matrix": c2w.tolist(),
#         "colmap_im_id": im_id,
    }
    frames.append(frame)
    
# Read camera params and write frame information
transforms_json = parse_colmap_camera_params(cam_id_to_camera[1])
transforms_json["frames"] = frames

# Scale camera params
cam_params = np.array([
    transforms_json["w"], transforms_json["h"], 
    transforms_json["fl_x"], transforms_json["fl_y"], transforms_json["cx"], transforms_json["cy"]])
cam_param_scaled = cam_params / pose_resize_scale

transforms_json["w"], transforms_json["h"], \
    transforms_json["fl_x"], transforms_json["fl_y"],\
    transforms_json["cx"], transforms_json["cy"] = tuple(cam_param_scaled)

write_to_json(root_dir/("transforms%s.json"%appendix), transforms_json)
print(f"Wrote info to {root_dir/('transforms%s.json'%appendix)} file!")

252 frames matched..
Wrote info to ../../data/real/ConferenceRoom/transforms_colmap.json file!
